# Creating your own fuzzer and fuzzing with it

---

**Student name:**  Nipun Waas

**Student number:** 2305414

---

This task has four sub-parts:

A) Designing a mutator that takes a valid URL as input and creates its mutations

B) Designing a target URL validator program and fuzz test it with mutations generated in task 1.

C) Creating a fuzzer

D) Generating mutations with Radamsa and observing program execution

#### A) Designing a mutator that takes valid url and creates mutations

In [28]:
# Creating the mutator
import uuid
import random
import string

''' Generating a single random string with a given length '''
def generate_random_single_string(stringLenth, onlyAscii = False):
    stringList = list(string.printable[:-5])
    if onlyAscii:
        stringList = stringList[:62]
    return ''.join(random.choice(stringList) for i in range(stringLenth))

''' 
Creating a random mutation for a given string. 
This includes
    1. Replacing character.
    2. Adding new characters in between or SOS or EOS.
    3. Generating a new string
'''
def generate_random_string(objectString, onlyAscii = False):
    generativeMutatedStringLength = 5
    mutationAction = random.randint(0, 2)
    stringList = list(string.printable[:-5])
    
    if onlyAscii:
        stringList = stringList[:62]
    # replacement
    if mutationAction == 0:
        randomIterStart = random.randint(0, len(objectString)-1)
        randomIterEnd = random.randint(randomIterStart, len(objectString)-1)
        for i in range(randomIterStart, randomIterEnd):
            objectString.replace(objectString[i], random.choice(stringList))
    # adding new characters
    elif mutationAction ==  1:
        randomIter = random.randint(0, len(objectString)-1)
        objectString = objectString[:randomIter] + ''.join(random.choice(stringList) for i in range(random.randint(0, generativeMutatedStringLength))) + objectString[randomIter:]
#     # create a new string
    elif mutationAction == 2:
        randomLength = random.randint(0, len(objectString) + generativeMutatedStringLength)
        objectString = generate_random_single_string(randomLength)
    return objectString

# Mutator function that takes a validated URL and creates random mutated URLs
def mutator(baseUrl):
    mutated_URL = ""
    # spliting the url by '/' and filtering out the white spaces
    splittedUrl = list(filter(lambda x: len(x) > 0, baseUrl.strip().split("/")))

    #scheme - choosing a random scheme from the given choices
    scheme = ["ftp://", "http://", "https://"]
    mutated_URL += random.choice(scheme)
    
    #netloc - creating a netloc with www. address and a generative host name
    netloc = "www." + generate_random_string(splittedUrl[1].split(".")[1])+ "." + generate_random_single_string(random.randint(2,4))
    mutated_URL += netloc
    
    #path - generate random path strings for each in the given url
    path = '/'.join([generate_random_string(i, True) for i in splittedUrl[2:-1]])
    mutated_URL += "/" + path + "/"
    
    # query
    querySplitted = splittedUrl[-1].split("?")
    mutated_URL += generate_random_single_string(len(querySplitted[0]), True) + "/"
    if len(querySplitted) > 1:
        queries = querySplitted[-1].split("=")
        for i in queries:
            mutated_URL += generate_random_single_string(len(i), True) + "="
    mutated_URL = mutated_URL[:-1]
    return mutated_URL

Running the mutator for 4 minutes and observes the results.

In [30]:
import time
'''
Running the mutator for 4 minutes
'''

t_end = time.time() + 60 * 4
mutatedInputCount = 0
mutated20Samples = []
while time.time() < t_end:
    if mutatedInputCount < 20: mutated20Samples.append(mutator("http://www.google.com/he/yey/search?q=fuzzing"))
    else: mutator("http://www.google.com/he/yey/search?q=fuzzing")
    mutatedInputCount += 1
print("Generated input count: " + str(mutatedInputCount))
print("Generated 20 samples: \n" + "\n".join(mutated20Samples))

Generated input count: 15894875
Generated 20 samples: 
ftp://www..Z~/{_BI/Qyey/j7ZZqJ/B=NmwAO4K
ftp://www.=.?U`/h2qDade/yVJt1ey/1tHROG/Z=3f5pZDc
http://www.google.#xLX/he/yejDsUy/1kliRv/M=fANnaJE
http://www.google.)8Nr/mW3he/tj|)@/lH5gEQ/i=fxOfxwu
http://www.google.u[/a/yey/3xjl8l/b=RznpPLb
http://www._k\Cqgo0}.Xh/^e/yey/5ZiqeI/x=NfJrnxK
http://www.googl|e.&4 !/he/Dr1yey/xR9J26/5=Te1jcnT
ftp://www.google.qlx/hne/yey/gnG11e/f=EnRPrMd
http://www.P1Y.{24/he/dE0\0^8/3vnYeD/O=FP0SXwn
http://www.google.f>I/ydOUhe/yey/XdbKsI/i=6IlyIho
ftp://www.google.uF_/.+k5/yey/2NGh7J/n=9vTjLWJ
https://www.google.!_/he/ye8fK78y/DeDAVu/A=oD6pICg
ftp://www.mnF>1[.sY+/he/yey/AnRROG/a=Jhimv54
ftp://www.vcBt5<~OK_B.MYm/>+C[`u$/yeXAgoLy/rjt82y/L=Eqj0KVB
ftp://www.0<|<x#-.'u/he/m/KQHndV/A=j1c7YK6
https://www.google.S*/5Wdhe/yey/TyTaAw/l=rZ1veIc
ftp://www.H,.&d?Q/he/yey/Kf0QIa/7=DQYwzbt
http://www.google.oV6/hrQMle/W/ZnkPks/u=bIR6YPM
http://www.go.ogle.r_A&/he/yej7WFoy/nYh63I/3=854FrGL
http://www.g*ZqD=oogle.4O8z/

#### B) Designing a target URL validator program and fuzz test it with mutations generated in task 1.

using ```validators``` library to validate the generated URL

In [6]:
import validators

def validator(url):
    validated = validators.url(url)
    if validated:
        return True
    return False

Testing the validator by providing the above 20 samples to check the program.

In [32]:
crashedCount = 0
crashedUrls = []

for url in mutated20Samples:
    if validator(url) :
        print("Successful url: ", url)
    else:
        print("Crashed url: ", url)

Crashed url:  ftp://www..Z~/{_BI/Qyey/j7ZZqJ/B=NmwAO4K
Crashed url:  ftp://www.=.?U`/h2qDade/yVJt1ey/1tHROG/Z=3f5pZDc
Crashed url:  http://www.google.#xLX/he/yejDsUy/1kliRv/M=fANnaJE
Crashed url:  http://www.google.)8Nr/mW3he/tj|)@/lH5gEQ/i=fxOfxwu
Crashed url:  http://www.google.u[/a/yey/3xjl8l/b=RznpPLb
Crashed url:  http://www._k\Cqgo0}.Xh/^e/yey/5ZiqeI/x=NfJrnxK
Crashed url:  http://www.googl|e.&4 !/he/Dr1yey/xR9J26/5=Te1jcnT
Successful url:  ftp://www.google.qlx/hne/yey/gnG11e/f=EnRPrMd
Crashed url:  http://www.P1Y.{24/he/dE0\0^8/3vnYeD/O=FP0SXwn
Crashed url:  http://www.google.f>I/ydOUhe/yey/XdbKsI/i=6IlyIho
Crashed url:  ftp://www.google.uF_/.+k5/yey/2NGh7J/n=9vTjLWJ
Crashed url:  https://www.google.!_/he/ye8fK78y/DeDAVu/A=oD6pICg
Crashed url:  ftp://www.mnF>1[.sY+/he/yey/AnRROG/a=Jhimv54
Crashed url:  ftp://www.vcBt5<~OK_B.MYm/>+C[`u$/yeXAgoLy/rjt82y/L=Eqj0KVB
Crashed url:  ftp://www.0<|<x#-.'u/he/m/KQHndV/A=j1c7YK6
Crashed url:  https://www.google.S*/5Wdhe/yey/TyTaAw/l=rZ1veIc

#### C) Creating a fuzzer

Here the fuzzer uses the previously created mutator and the validator to perform mutations on the validator(targeted program)

In [7]:
import os

hostname = "http://www.google.com/he/yey/search?q=fuzzing"
crashedUrlsForCases = {}
crashedCountsForCases = {}

for testCasesCount in [100, 1000, 10000]:
    mutatedUrls = [mutator(hostname) for i in range(testCasesCount)]

    crashedCount = 0
    crashedUrls = []

    for url in mutatedUrls:
        if validator(url) == False:
            crashedCount+=1
            crashedUrls.append(url)
            
    crashedUrlsForCases[testCasesCount] = crashedUrls
    crashedCountsForCases[testCasesCount] = crashedCount
    print("Test Case: " + str(testCasesCount) + " Crashed count: " + str(crashedCount))

Test Case: 100 Crashed count: 88
Test Case: 1000 Crashed count: 880
Test Case: 10000 Crashed count: 8873


Some samples of the crashed inputs

In [38]:
for key in crashedUrlsForCases.keys():
    print("Crashed URLs for test case: " + str(key) + "\n" + "\n".join(crashedUrlsForCases[key][:5]))

Crashed URLs for test case: 100
ftp://www.googl^M?_e.U[`*/hG2xGe/yey/6OQoYz/m=YHTAHSb
ftp://www.google.i'_F/)U/FX/2Cadoz/E=DGKDJ9x
https://www.gook4@&:gle.Q*(f/d[~n/GbVEyey/wbmlgR/d=L5c2Uqc
http://www.Fl&U{2-fn.?=|R/ohe/yrTLey/VRtWiB/K=ODIDvQZ
ftp://www.U}!/:Ea|_`.bz/he/54clyey/s3YThV/a=ijslPbB
Crashed URLs for test case: 1000
https://www.a'GD~J~ >."Z/0fqRhe/yey/gBihxu/8=Zk7Torz
https://www.ogoogle.z--/$:!%u/yey/TOZOLD/E=K0Z3wVm
http://www.Z.\zY//yey/kdwasd/D=AbSwwnk
ftp://www.YSX%kRK.qP/2Xhe/yey/W4pnDk/y=mhW9SQu
https://www.googlde.G>/$>G2/yey/TYO9Ty/n=2YRTGHB
Crashed URLs for test case: 10000
ftp://www..v.r^bg/he/yey/FQAMXj/6=PCBYJTL
ftp://www.googXRu`Qle.qEX/he//s1KCxv/w=AAgaGeO
http://www.go4G~yiogle.Zs/8ZFOEhe/yey/OiSlmM/E=CZPBdgJ
https://www.google.Qqx=/hT7ZmEe/ynlp1ey/J6XSRR/8=apTPFsS
http://www.google.0"/he/->/4meMk9/U=9YgeCKr


### D) Generating mutations with Radamsa and observing program execution
Here the fuzzer reads a text file that cnotains malformed inputs that are generated using radamsa. it will execute against the validator and look for the crashs.

In [43]:
import os

decodedCount = 0
crashes = 0
crashedUrls = []
successUrls = []

testCases = [100, 1000, 10000]

with open("output.txt", "rb") as f:
    lines = f.readlines()
    for i, line in enumerate(lines):
        try:
            decodedMutation = line.decode('utf-8')
        except:
            continue
        validatedUrl = validator(decodedMutation[:-1])
        if validatedUrl == False:
            crashedUrls.append(decodedMutation)
            crashes += 1
        else:
            successUrls.append(decodedMutation)
        if decodedCount in testCases:
            testCases.remove(decodedCount)
            print("Test Case: " + str(decodedCount) + " Crashed count: " + str(crashes))
            decodedCount = 0
            if len(testCases) == 0:
                break
            continue
        decodedCount += 1
               

Test Case: 100 Crashed count: 7
Test Case: 1000 Crashed count: 594
Test Case: 10000 Crashed count: 6887


Few successfully invoked urls and crashed urls

In [50]:
for i in successUrls[:10]:
    print("Successfull Urls: "+ i)

Successfull Urls: http://www.google.com/search?q=fuzzing

Successfull Urls: http://www.google.com/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzziom/search?q=fuzzing

Successfull Urls: http://www.google.com/rearchf?q=uzzinww.google.com/rearchf?q=uzzing

Successfull Urls: http://www.google.com/search?q=fuzzing

Successfull Urls: http://www.google.com/search?q=fuzzing

Successfull Urls: 

In [51]:
for i in crashedUrls[:10]:
    print("Crashed Urls: "+ i)

Crashed Urls: g

Crashed Urls: ?q=fuzzinffoogle.cohttp://www.google.com/search?q=fuzzinh

Crashed Urls: http://www.goo󠁻gle.com/search?q=fuzzing

Crashed Urls: http:󠁞//www.google.c󠁂mtsearch?q=fuzzing

Crashed Urls: http://󠀮w󠁥ww.google.com.s‪earc?q󠀸=󠁿fuzzing

Crashed Urls: http://www.google.com/searc󠁋h?q=fuzzing

Crashed Urls: http://www.google.co󠁼m/search?q=fuzzing

Crashed Urls: http://www.googl󠁊e.com/search?q=fuzzing

Crashed Urls: hutp󠁏://www.google.com/search?q=fuzzing

Crashed Urls: http://www.google.com/rearch?q=
uzzing

